In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [3]:
 ## Task 1: Mark all launch sites on a map




First, let's try to add each site's location on a map using site's latitude and longitude coordinates







The following dataset with the name spacex_launch_geo.csv is an augmented dataset with latitude and longitude added for each site.

In [5]:
import requests
import io
# URL of the CSV file
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Download the CSV file using requests
response = requests.get(URL)
spacex_csv_file = io.BytesIO(response.content)

# Read the CSV file into a Pandas DataFrame
spacex_df = pd.read_csv(spacex_csv_file)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.

In [6]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use folium.Circle to add a highlighted circle area with a text label on a specific coordinate. For example,

In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

 let's add a circle for each launch site in data frame launch_sites

TODO: Create and add folium.Circle and folium.Marker for each launch site on the site map

An example of folium.Circle:

folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))

An example of folium.Marker:

folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))

In [16]:

# ... (Your existing code for loading the DataFrame) ...

# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)  # Adjust zoom level as needed

# Iterate through the launch sites DataFrame
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    # Create a Circle object for each launch site
    circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(row['Launch Site']))
    # Create a Marker object for each launch site
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    )
    # Add the Circle and Marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map

In [17]:


# Now let's try to mark the success/failed launches for each site on the map

marker_cluster = MarkerCluster()

# Iterate through the spacex_df DataFrame
for index, row in spacex_df.iterrows():
    # Assign colors based on launch success/failure
    if row['class'] == 1:
        marker_color = 'green'  # Success
    else:
        marker_color = 'red'  # Failure

    # Create a marker for each launch site
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=marker_color),
        popup=row['Launch Site']
    )
    marker_cluster.add_child(marker)


site_map.add_child(marker_cluster)
site_map

In [18]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [19]:
marker_cluster = MarkerCluster()

In [20]:
# prompt: # Apply a function to check the value of `class` column
# # If class=1, marker_color value will be green
# # If class=0, marker_color value will be red

# ... (Your existing code) ...

# Now let's try to mark the success/failed launches for each site on the map

marker_cluster = MarkerCluster()

# Iterate through the spacex_df DataFrame
for index, row in spacex_df.iterrows():
    # Assign colors based on launch success/failure
    if row['class'] == 1:
        marker_color = 'green'  # Success
    else:
        marker_color = 'red'  # Failure

    # Create a marker for each launch site
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=marker_color),
        popup=row['Launch Site']
    )
    marker_cluster.add_child(marker)


site_map.add_child(marker_cluster)
site_map

In [21]:
# prompt: # Add marker_cluster to current site_map
# site_map.add_child(marker_cluster)
# # for each row in spacex_df data frame
# # create a Marker object with its coordinate
# # and customize the Marker's icon property to indicate if this launch was successed or failed,
# # e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
# for index, record in spacex_df.iterrows():
#     # TODO: Create and add a Marker cluster to the site map
#     # marker = folium.Marker(...)
#     marker_cluster.add_child(marker)
# site_map

# ... (Your existing code) ...

marker_cluster = MarkerCluster()

# Iterate through the spacex_df DataFrame
for index, record in spacex_df.iterrows():
    # Assign colors based on launch success/failure
    if record['class'] == 1:
        marker_color = 'green'  # Success
    else:
        marker_color = 'red'  # Failure

    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=marker_color),  # Customize icon
        popup=record['Launch Site']
    )

    marker_cluster.add_child(marker)

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

site_map

In [22]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [23]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [24]:

# Assuming you have a list of coastline coordinates (latitude, longitude)
coastline_coordinates = [
    [30.0, -80.0],  # Example coastline coordinate 1
    [29.5, -81.0],  # Example coastline coordinate 2
    # Add more coastline coordinates as needed
]

launch_site_lat = 28.56367
launch_site_lon = -80.57163

min_distance = float('inf')
closest_coastline_coordinate = None

for coastline_lat, coastline_lon in coastline_coordinates:
    distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
    if distance_coastline < min_distance:
        min_distance = distance_coastline
        closest_coastline_coordinate = [coastline_lat, coastline_lon]

print("Closest coastline coordinate:", closest_coastline_coordinate)
print("Distance to closest coastline:", min_distance)

Closest coastline coordinate: [29.5, -81.0]
Distance to closest coastline: 112.17085905912424


In [25]:


# Assuming you have a list of coastline coordinates (latitude, longitude)
coastline_coordinates = [
    [28.56367, -80.57163],  # Example coastline coordinate 1
    # Add more coastline coordinates as needed
]

launch_site_lat = 28.56367
launch_site_lon = -80.57163

min_distance = float('inf')
closest_coastline_coordinate = None

for coastline_lat, coastline_lon in coastline_coordinates:
    distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
    if distance_coastline < min_distance:
        min_distance = distance_coastline
        closest_coastline_coordinate = [coastline_lat, coastline_lon]

print("Closest coastline coordinate:", closest_coastline_coordinate)
print("Distance to closest coastline:", min_distance)


# Create and add a folium.Marker on your selected closest coastline point on the map
if closest_coastline_coordinate:
    distance_marker = folium.Marker(
        closest_coastline_coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(min_distance),
        )
    )
    site_map.add_child(distance_marker)

site_map

Closest coastline coordinate: [28.56367, -80.57163]
Distance to closest coastline: 0.0


In [26]:

# Assuming you have a list of coastline coordinates (latitude, longitude)
coastline_coordinates = [
    [28.56367, -80.57163],  # Example coastline coordinate 1
    # Add more coastline coordinates as needed
]

launch_site_lat = 28.56367
launch_site_lon = -80.57163

min_distance = float('inf')
closest_coastline_coordinate = None

for coastline_lat, coastline_lon in coastline_coordinates:
    distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
    if distance_coastline < min_distance:
        min_distance = distance_coastline
        closest_coastline_coordinate = [coastline_lat, coastline_lon]

print("Closest coastline coordinate:", closest_coastline_coordinate)
print("Distance to closest coastline:", min_distance)


# Create and add a folium.Marker on your selected closest coastline point on the map
if closest_coastline_coordinate:
    distance_marker = folium.Marker(
        closest_coastline_coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(min_distance),
        )
    )
    site_map.add_child(distance_marker)

site_map

Closest coastline coordinate: [28.56367, -80.57163]
Distance to closest coastline: 0.0


In [27]:
# prompt: # Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# # lines=folium.PolyLine(locations=coordinates, weight=1)
# site_map.add_child(lines)

# Assuming you have the launch site coordinates and coastline coordinates
launch_site_lat = 28.56367
launch_site_lon = -80.57163
coastline_coordinates = [
    [28.56367, -80.57163],  # Example coastline coordinate
    # Add more coastline coordinates as needed
]

# Create a PolyLine object
coordinates = [[launch_site_lat, launch_site_lon], coastline_coordinates[0]]  # Assuming only one coastline coordinate
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [ ]:
# prompt: # Create a marker with distance to a closest city, railway, highway, etc.
# # Draw a line between the marker to the launch site

# ... (Your existing code) ...


# Assuming you have the launch site coordinates and coastline coordinates
launch_site_lat = 28.56367
launch_site_lon = -80.57163
coastline_coordinates = [
    [28.56367, -80.57163],  # Example coastline coordinate
    # Add more coastline coordinates as needed
]

# Create a PolyLine object
if coastline_coordinates:
    coordinates = [[launch_site_lat, launch_site_lon], coastline_coordinates[0]]  # Assuming only one coastline coordinate
    lines = folium.PolyLine(locations=coordinates, weight=1)
    site_map.add_child(lines)

# Display the map
site_map

In [28]:

# Assuming you have the launch site coordinates and relevant proximity coordinates (railways, highways, coastline, cities)
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]


# Example proximity coordinates (replace with your actual data)
railway_coordinates = [
    [28.57, -80.58],  # Example railway coordinate
    # Add more railway coordinates as needed
]
highway_coordinates = [
    [28.56, -80.57],  # Example highway coordinate
    # Add more highway coordinates as needed
]
coastline_coordinates = [
    [28.56, -80.56],  # Example coastline coordinate
    # Add more coastline coordinates as needed
]
city_coordinates = [
    [28.60, -80.60],  # Example city coordinate
    # Add more city coordinates as needed
]


def plot_distance_lines(site_map, launch_site_lat, launch_site_lon, proximity_coordinates, line_color):
    """
    Plots distance lines from a launch site to a list of proximity coordinates.

    Args:
        site_map: The folium map object.
        launch_site_lat: Latitude of the launch site.
        launch_site_lon: Longitude of the launch site.
        proximity_coordinates: A list of coordinates representing proximity points.
        line_color: The color of the line to be drawn.
    """
    for proximity_lat, proximity_lon in proximity_coordinates:
        coordinates = [[launch_site_lat, launch_site_lon], [proximity_lat, proximity_lon]]
        lines = folium.PolyLine(locations=coordinates, weight=1, color=line_color)
        site_map.add_child(lines)


# Iterate through launch sites and plot distance lines
for index, row in launch_sites_df.iterrows():
    launch_site_lat = row['Lat']
    launch_site_lon = row['Long']

    # Plot lines to railways (example)
    plot_distance_lines(site_map, launch_site_lat, launch_site_lon, railway_coordinates, 'blue')

    # Plot lines to highways (example)
    plot_distance_lines(site_map, launch_site_lat, launch_site_lon, highway_coordinates, 'red')

    # Plot lines to coastline (example)
    plot_distance_lines(site_map, launch_site_lat, launch_site_lon, coastline_coordinates, 'green')

    # Plot lines to cities (example)
    plot_distance_lines(site_map, launch_site_lat, launch_site_lon, city_coordinates, 'purple')


# Display the map
site_map


# Answer the questions based on the plotted lines:

# Are launch sites in close proximity to railways?
# Look for short blue lines connecting launch sites to railway coordinates.

# Are launch sites in close proximity to highways?
# Look for short red lines connecting launch sites to highway coordinates.

# Are launch sites in close proximity to coastline?
# Look for short green lines connecting launch sites to coastline coordinates.

# Do launch sites keep certain distance away from cities?
# Look for the length of purple lines connecting launch sites to city coordinates.
# If the lines are relatively long, it indicates that launch sites maintain a distance from cities.
